# Word2vec

In [1]:
# 부모 폴더의 경로를 추가
import sys; sys.path.insert(0, '..')

from util.data_loader import DataLoader
from util.metric_calculator import MetricCalculator

In [2]:
# Movielens 데이터 로딩
data_loader = DataLoader(num_users=1000, num_test_items=5, data_path='../data/ml-10M100K/')
movielens = data_loader.load()

In [3]:
import gensim
import logging

movie_content = movielens.item_content.copy()

# tag가 부여되어 있지 않은 영화가 있지만. genre는 모든 영화에 부여되어 있다
# tag와 genre를 결합한 것을 영화의 콘텐츠 정보로 해서 비슷한 영화를 찾아서 추천한다
# tag가 없는 영화에서는 NaN으로 되어 있으므로, 빈 리스트로 초기화한다
movie_content['tag_genre'] = movie_content['tag'].fillna("").apply(list) + movie_content['genre'].apply(list)
movie_content['tag_genre'] = movie_content['tag_genre'].apply(lambda x:set(map(str, x)))

# 태그와 장르 데이터를 사용해서, word2vec를 학습한다
tag_genre_data = movie_content.tag_genre.tolist()
model = gensim.models.word2vec.Word2Vec(tag_genre_data, vector_size=100, window=100, sg=1, hs=0, epochs=50, min_count=5)


In [4]:
# anime 태그와 비슷한 태그를 확인한다
model.wv.most_similar('anime')

[('studio ghibli', 0.8056841492652893),
 ('zibri studio', 0.8047590851783752),
 ('pelicula anime', 0.7545369863510132),
 ('hayao miyazaki', 0.7496048212051392),
 ('miyazaki', 0.7415587306022644),
 ('japan', 0.6943755149841309),
 ('Animation', 0.5248773694038391),
 ('curse', 0.48674505949020386),
 ('girl', 0.48163750767707825),
 ('steampunk', 0.4724525511264801)]

In [5]:
# Word2vecContent 추천
from src.word2vec import Word2vecRecommender
recommender = Word2vecRecommender()
recommend_result = recommender.recommend(movielens)

In [6]:
# 평가
metric_calculator = MetricCalculator()
metrics = metric_calculator.calc(
    movielens.test.rating.tolist(), recommend_result.rating.tolist(),
    movielens.test_user2items, recommend_result.user2items, k=10)
print(metrics)

rmse=0.000, Precision@K=0.011, Recall@K=0.035
